In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, Input
from keras import applications
from keras.layers import AveragePooling2D, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, CSVLogger, TensorBoard, LambdaCallback
from keras.applications.densenet import DenseNet201, preprocess_input 
from keras.layers import Conv2D, Convolution2D, MaxPooling2D, ZeroPadding2D, BatchNormalization, Activation
from keras.optimizers import Adam
from keras import backend as K
import numpy as np
import pandas as pd
from keras import layers
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.models import load_model
import random
import cv2
from PIL import Image
import os
from keras.utils import to_categorical
from collections import defaultdict
import jpeg4py as jpeg
from io import BytesIO

Using TensorFlow backend.


In [2]:
#parameters
classes = 10
batch_size=40
image_size = 512
train_total = 3500
validation_total = 875

In [3]:
model_densenet = DenseNet201(include_top=False, weights = None,input_shape=(image_size,image_size,3))

In [5]:
x = model_densenet.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
x = Dense(classes, activation='softmax')(x)

In [6]:
for layer in model_densenet.layers:
    layer.trainable = True

In [8]:
model = Model(inputs=model_densenet.input, outputs=x)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 518, 518, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 256, 256, 64) 9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 256, 256, 64) 256         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu

In [9]:
train_datagen = ImageDataGenerator(horizontal_flip=True)
validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
                    

In [91]:
train_generator = train_datagen.flow_from_directory(
                    'train',
                    target_size=(image_size,image_size),
                    batch_size=batch_size,
                    class_mode='categorical',
                    shuffle=True
                    )


Found 906500 images belonging to 10 classes.


In [10]:
validation_generator = validation_datagen.flow_from_directory(
                        'validation',
                        target_size=(image_size,image_size),
                        batch_size=batch_size,
                        class_mode='categorical',
                        shuffle=False)

Found 875 images belonging to 10 classes.


In [143]:
def image_aug_valid(image, index):
    Image.LOAD_TRUNCATED_IMAGES = True
    mpimg.LOAD_TRUNCATED_IMAGES = True
    if(index==0):
        res_image = np.uint8(cv2.pow(image/255.,0.8)*255.)
    elif(index==1):
        res_image = np.uint8(cv2.pow(image/255.,1.2)*255.)
    elif(index==2):
        res_image = cv2.resize(image,(0,0),fx=0.5,fy=0.5,interpolation = cv2.INTER_CUBIC)
    elif(index==3):
        res_image = cv2.resize(image,(0,0),fx=0.8,fy=0.8,interpolation = cv2.INTER_CUBIC)    
    elif(index==4):
        res_image = cv2.resize(image,(0,0),fx=1.5,fy=1.5,interpolation = cv2.INTER_CUBIC)
    elif(index==5):
        res_image = cv2.resize(image,(0,0),fx=2.0,fy=2.0,interpolation = cv2.INTER_CUBIC)
    elif(index==6):
        img = Image.fromarray(image)
        out_70_valid = BytesIO()
        img.save(out_70_valid, "JPEG", quality=70)
        res_image = jpeg.JPEG(np.frombuffer(out_70_valid.getvalue(), dtype=np.uint8)).decode()
        del img
        del out_70_valid
    elif(index==7):
        img = Image.fromarray(image)
        out_90_valid = BytesIO()
        img.save(out_90_valid, "JPEG", quality=90)
        res_image = jpeg.JPEG(np.frombuffer(out_90_valid.getvalue(), dtype=np.uint8)).decode()
        del img
        del out_90_valid
    return res_image 

In [144]:
def image_aug(image, index):
    Image.LOAD_TRUNCATED_IMAGES = True
    mpimg.LOAD_TRUNCATED_IMAGES = True
    if(index==0):
        res_image = np.uint8(cv2.pow(image/255.,0.8)*255.)
    elif(index==1):
        res_image = np.uint8(cv2.pow(image/255.,1.2)*255.)
    elif(index==2):
        res_image = cv2.resize(image,(0,0),fx=0.5,fy=0.5,interpolation = cv2.INTER_CUBIC)
    elif(index==3):
        res_image = cv2.resize(image,(0,0),fx=0.8,fy=0.8,interpolation = cv2.INTER_CUBIC)    
    elif(index==4):
        res_image = cv2.resize(image,(0,0),fx=1.5,fy=1.5,interpolation = cv2.INTER_CUBIC)
    elif(index==5):
        res_image = cv2.resize(image,(0,0),fx=2.0,fy=2.0,interpolation = cv2.INTER_CUBIC)
    elif(index==6):
        img = Image.fromarray(image)
        out_70 = BytesIO()
        img.save(out_70, "JPEG", quality=70)
        res_image = jpeg.JPEG(np.frombuffer(out_70.getvalue(), dtype=np.uint8)).decode()
        del img
        del out_70
    elif(index==7):
        img = Image.fromarray(image)
        out_90 = BytesIO()
        img.save(out_90, "JPEG", quality=90)
        res_image = jpeg.JPEG(np.frombuffer(out_90.getvalue(), dtype=np.uint8)).decode()
        del img
        del out_90
    return res_image 

In [145]:
def random_crop(image, crop_size):
    h,w,d = image.shape
    rand_num_h = random.randint(0,h-crop_size)
    rand_num_w = random.randint(0,w-crop_size)
    image_crop = image[rand_num_h:rand_num_h+crop_size,rand_num_w:rand_num_w+crop_size,:]
    return image_crop

In [146]:
def train_gen(train_files, train_classes, batch_size, target_size, imggen):
    min_batch_size = batch_size //10
    train_data = list(zip(train_files, train_classes))
    train_data_dict = defaultdict(list)
    for i in range(len(train_data)):
        train_data_dict[train_data[i][0].split('/')[0]].append(train_data[i])
       
    while(True):
        for i in train_data_dict.keys():
            random.shuffle(train_data_dict[i])
        for start in range(0, len(train_data), min_batch_size):
            image_crop_list = []
            image_classes_list = []
            
            
            for j in train_data_dict.keys():
                end = min(start + min_batch_size, len(train_data_dict[j])) 
                
                x_batch = [train_data_dict[j][i][0] for i in range(start,end)]
                
                for index,i in enumerate(x_batch):
                    rand_num = random.randint(0,7)
                    image = jpeg.JPEG('train/' + i).decode()
                    
                    if(len(image.shape)<3):
                        print(_1_)
                        continue
                    
                    
                    image = image_aug(image,rand_num)
                    image = random_crop(image,image_size)
                    if(np.random.rand()<0.1):
                        image = np.rot90(image,1,(0,1))
                    image_crop_list.append(image)
                    image_classes_list.append(train_data_dict[j][start+index][1])
                    
                    del image
                    
            
            if(len(image_classes_list)<batch_size-2*min_batch_size):
                break
            image_crop_list = [x/1. for x in image_crop_list]
            image_crop_list = [imggen.random_transform(x) for x in image_crop_list]
            image_crop_list = [preprocess_input(x) for x in image_crop_list]

            x_batch = np.array(image_crop_list, np.float32)
            y_batch = np.array(image_classes_list)         
            
            yield (x_batch, y_batch)

In [147]:
train_crop_generator = train_gen(train_generator.filenames,to_categorical(train_generator.classes),batch_size,image_size,train_datagen) 

In [289]:
def valid_gen(valid_files, valid_classes, batch_size, target_size, imggen):

    valid_data = list(zip(valid_files, valid_classes))
  
    while(True):
        
        for manip_num in range(8):
            for start in range(0, len(valid_data), batch_size):
                image_crop_list = []
                image_classes_list = []
                end = min(start + batch_size, len(valid_data)) 

                x_batch = [valid_data[i][0] for i in range(start,end)]

                for index,i in enumerate(x_batch):
                    #if(manip_num==2 or manip_num==3):
                    #    temp_file = random.choice(os.listdir('validation_1024/'+i.split('/')[0]))
                    #    image = jpeg.JPEG('validation_1024/' + i.split('/')[0]+'/' + temp_file).decode()
                    #else:
                    image = jpeg.JPEG('validation/' + i).decode()
                    if(len(image.shape)<3):
                            continue
                    #rand_num = random.randint(0,7)

                    image = image_aug_valid(image,manip_num)
                    image = random_crop(image,image_size)
                    image_crop_list.append(image)
                    image_classes_list.append(valid_data[start+index][1])

                    del image

                image_crop_list = [x/1. for x in image_crop_list]
                image_crop_list = [preprocess_input(x) for x in image_crop_list]

                x_batch = np.array(image_crop_list, np.float32)
                y_batch = np.array(image_classes_list)         

                yield (x_batch, y_batch)

In [290]:
valid_aug_generator = valid_gen(validation_generator.filenames,to_categorical(validation_generator.classes),batch_size,image_size,validation_datagen) 

In [11]:
model.compile(loss='categorical_crossentropy',
             optimizer=Adam(lr=1e-5),
             metrics=['accuracy'])

In [151]:
from pushbullet import Pushbullet
pb = Pushbullet('o.KiDDDXPuzV4qKbXh4Lywbgw1tK2oFfq1')

In [152]:
pushbullet_callback = LambdaCallback(
    on_epoch_end=lambda epoch, logs: pb.push_note("epoch: "+str(epoch),"train_loss: "+str(logs['loss'])+"    val_loss"+str(logs['val_loss'])))

In [153]:
callbacks = [ModelCheckpoint(filepath='densenet_unalt_4.hdf5', verbose=1, save_best_only=True, save_weights_only=True),
ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1,epsilon=0.01),
#EarlyStopping(monitor='val_loss', patience=10, verbose=1),
CSVLogger('./3-metrics_densenet_unalt_1.csv'),
TensorBoard(log_dir='logs_densenet_unalt', write_graph=True)]

In [133]:
history = model.fit_generator(
                    train_crop_generator,
                    steps_per_epoch = 10*int(np.ceil(train_total/batch_size)),
                    epochs=100,
                    validation_data=valid_aug_generator,
                    validation_steps= 4*int(np.ceil(validation_total/batch_size)),
                    verbose=1,
                    callbacks=callbacks)

Epoch 1/100
879/880 [============================>.] - ETA: 0s - loss: 0.0111 - acc: 0.9977
Epoch 00001: val_loss improved from inf to 0.19214, saving model to densenet_unalt_4.hdf5
880/880 [==============================] - 552s 627ms/step - loss: 0.0111 - acc: 0.9977 - val_loss: 0.1921 - val_acc: 0.9634
Epoch 2/100
879/880 [============================>.] - ETA: 0s - loss: 0.0125 - acc: 0.9974
Epoch 00002: val_loss did not improve
880/880 [==============================] - 537s 610ms/step - loss: 0.0125 - acc: 0.9974 - val_loss: 0.2147 - val_acc: 0.9623
Epoch 3/100
879/880 [============================>.] - ETA: 0s - loss: 0.0111 - acc: 0.9977
Epoch 00003: val_loss did not improve
880/880 [==============================] - 538s 611ms/step - loss: 0.0111 - acc: 0.9977 - val_loss: 0.2349 - val_acc: 0.9591
Epoch 4/100
879/880 [============================>.] - ETA: 0s - loss: 0.0120 - acc: 0.9975
Epoch 00004: val_loss did not improve

Epoch 00004: ReduceLROnPlateau reducing learning rat

KeyboardInterrupt: 

In [12]:
model.load_weights('densenet_image_aug_3.hdf5')

In [293]:
model.evaluate_generator(valid_aug_generator,int(np.ceil(validation_total/batch_size)))

[0.090087271205016542, 0.97600000381469731]

In [1]:
# validation predictions for future ensembling

In [252]:
densenet_valid_preds = model.predict_generator(valid_aug_generator,int(np.ceil(validation_total/batch_size)))

In [255]:
valid_prev_filenames = np.load('filenames_list.npy')

In [265]:
np.save('densenet_valid_preds.npy',corrected_densenet_valid_preds)

In [33]:
# validation predictions for analysis

In [34]:
predictions_valid = model.predict_generator(validation_generator,int(np.ceil(validation_total/batch_size)))
predictions_valid = np.argmax(predictions_valid, axis=1)

In [35]:
validation_generator.class_indices

{'HTC-1-M7': 0,
 'LG-Nexus-5x': 1,
 'Motorola-Droid-Maxx': 2,
 'Motorola-Nexus-6': 3,
 'Motorola-X': 4,
 'Samsung-Galaxy-Note3': 5,
 'Samsung-Galaxy-S4': 6,
 'Sony-NEX-7': 7,
 'iPhone-4s': 8,
 'iPhone-6': 9}

In [36]:
true_positive = np.zeros(10)
false_positive = np.zeros(10)
true_negative = np.zeros(10)
false_negative = np.zeros(10)
for i in range(len(predictions_valid)):
    if(predictions_valid[i]==validation_generator.classes[i]):
        true_positive[predictions_valid[i]] = true_positive[predictions_valid[i]] + 1
    else:
        false_positive[predictions_valid[i]] = false_positive[predictions_valid[i]] + 1
        false_negative[validation_generator.classes[i]] = false_negative[validation_generator.classes[i]] +1

In [37]:
print('true_positive')
print(true_positive)
print('false_positive')
print(false_positive)
print('false_negative')
print(false_negative)
print('Total')
print(true_positive+false_negative)


true_positive
[ 86.  72.  53.  53.  62.  64.  82.  68.  68.  81.]
false_positive
[ 16.  27.   5.  19.  31.  22.  12.   7.   8.  39.]
false_negative
[ 12.  27.  22.  24.  24.  12.  17.  18.  20.  10.]
Total
[ 98.  99.  75.  77.  86.  76.  99.  86.  88.  91.]


In [2]:
#test predictions - 512 size

In [13]:
filenames_list = []
images_list = []
for i in os.listdir('test/temp/'):
    img = Image.open('test/temp/'+i)
    filenames_list.append(i)
    images_list.append(np.array(img, np.float32))
    

In [14]:
images_list = [np.rot90(x,3,(0,1)) for x in images_list]
images_list = [preprocess_input(x) for x in images_list]

In [15]:
new_array_arr = np.array(images_list)


In [16]:
new_array_arr.shape

(2640, 512, 512, 3)

In [3]:
# predicting test set with different orientations (Test Time Augmentation)

In [298]:
test_preds_1 = model.predict(new_array_arr)

In [299]:
np.save('densenet_512_test_preds_1.npy',test_preds_1)

In [304]:
test_preds_2 = model.predict(new_array_arr)

In [305]:
np.save('densenet_512_test_preds_2.npy',test_preds_2)

In [17]:
test_preds_3 = model.predict(new_array_arr)

In [18]:
np.save('densenet_512_test_preds_3.npy',test_preds_3)

In [4]:
# averageing mobilenet and densenet test results with TTA

In [19]:
mobilenet_test_preds_1 = np.load('mobilenet_512_test_preds_1.npy')
mobilenet_test_preds_2 = np.load('mobilenet_512_test_preds_2.npy')
mobilenet_test_preds_3 = np.load('mobilenet_512_test_preds_3.npy')

In [20]:
densenet_test_preds_1 = np.load('densenet_512_test_preds_1.npy')
densenet_test_preds_2 = np.load('densenet_512_test_preds_2.npy')
densenet_test_preds_3 = np.load('densenet_512_test_preds_3.npy')

In [21]:
final_test_preds_mean = (mobilenet_test_preds_1+mobilenet_test_preds_2+mobilenet_test_preds_3+densenet_test_preds_1+densenet_test_preds_2+densenet_test_preds_3)/6

In [23]:
final_test_preds_mean.shape

(2640, 10)

In [24]:
final_test_preds_mean_classes = np.argmax(final_test_preds_mean,axis=1)

In [215]:
final_test_preds_classes = []

In [218]:
for i in range(len(filenames_list)):
    final_test_preds_classes.append(np.argmax(np.bincount(np.array([test_preds_1_classes[i],test_preds_2_classes[i],test_preds_3_classes[i]]))))

In [223]:
np.bincount(final_test_preds_classes==test_preds_3_classes)

array([  14, 2626])

In [224]:
test_preds_mean = (test_preds_1+test_preds_2+test_preds_3)/3

In [233]:
np.bincount(np.argmax(test_preds_mean,axis=1)==final_test_preds_classes)

array([   6, 2634])

In [236]:
len(filenames_list)

2640

In [29]:
test_preds_unalt = model.predict(new_array_arr)

In [31]:
predicted_classes_manip = [class_ids[np.argmax(i)] for i in test_preds]
predicted_classes_unalt = [class_ids[np.argmax(i)] for i in test_preds_unalt]

In [31]:
class_ids = {validation_generator.class_indices[x]: x for x in validation_generator.class_indices}

In [53]:
model.load_weights('resnet_image_aug_full.hdf5')

In [63]:
model.evaluate_generator(valid_aug_generator,8*int(np.ceil(validation_total/batch_size)))

[0.35670383436871428, 0.90357143240315574]

In [ ]:
test_preds_unalt = model.predict(new_array_arr)

In [74]:
test_preds_avg = (test_preds+test_preds_unalt)/2

In [32]:
predicted_classes = [class_ids[i] for i in final_test_preds_mean_classes]

In [6]:
# combining unalt model test results with manip model test results

In [35]:
for index,i in enumerate(filenames_list):
    if(i.split('_')[-1]=='manip.tif'): 
        predicted_classes_unalt[index]=predicted_classes_manip[index]

In [284]:
final_test_preds_classes = [class_ids[i] for i in test_preds_1_classes]

In [36]:
submission = pd.DataFrame({'fname':filenames_list,'camera':predicted_classes})
submission.to_csv('submission_densenet_mobilenet_avg_tta_full_sub.csv', encoding="utf8", index=False)

In [37]:
from IPython.display import FileLink
FileLink('submission_densenet_mobilenet_avg_tta_full_sub.csv')

/home/janardhan/camera_model/submission_densenet_mobilenet_avg_tta_full_sub.csv

In [ ]:
# Private LB scores - 
# only DenseNet - 97.22
# mobilenet+densenet avg - 97.46